# Summaise depth maps

In [1]:
import yaml
from astropy.table import Table, vstack, unique
import numpy as np
import glob

In [2]:
meta_files = glob.glob('../*/meta_main.yml')

In [3]:
depths  =Table()

In [4]:
depths['field'] = [yaml.load(open(y, 'r'))['field'] for y in meta_files]

/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [5]:
bands = ['u', 'g', 'r', 'i', 'z', 'y', 'j', 'h', 'k', 'ks', 
         'irac_i1', 'irac_i2', 'irac_i3', 'irac_i4', 
          'mips_24', 'pacs_green', 'pacs_red', 
         'spire_250', 'spire_350', 'spire_500']

In [6]:
for band in bands:
    depths['ferr_{}_mean_median'.format(band)] = np.full(len(depths), np.nan)

In [7]:
for y in meta_files:
    yml = yaml.load(open(y, 'r'))
    try:
        depth = Table.read(
            "../dmu32_{}/data/depths_{}_{}.fits".format(
            yml['field'].replace('HATLAS-',''), 
            yml['field'].lower(), 
            yml['final'].split('.')[0].split('_')[-1] )
        )
    except:
        print('No depth present on ' + yml['field'])
        continue
    for band in bands:
        try:
            depths['ferr_{}_mean_median'.format(band)][
                depths['field']==yml['field']
            ] = np.nanmedian(depth['ferr_{}_mean'.format(band)])
        except:
            print(band, ' missing')

/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


No depth present on COSMOS
No depth present on ELAIS-S1
No depth present on GAMA-09
No depth present on Lockman-SWIRE
No depth present on ELAIS-N2
No depth present on Herschel-Stripe-82
No depth present on SA13
No depth present on GAMA-12
No depth present on GAMA-15
No depth present on AKARI-SEP
No depth present on HATLAS-SGP
No depth present on xFLS
No depth present on CDFS-SWIRE
No depth present on SSDF
No depth present on XMM-LSS
No depth present on SPIRE-NEP
h  missing
k  missing
irac_i3  missing
irac_i4  missing
No depth present on ELAIS-N1
No depth present on HATLAS-NGP
No depth present on XMM-13hr
No depth present on Bootes
No depth present on HDF-N
No depth present on EGS


In [8]:
"../dmu32_{}/data/depths_{}_{}.fits".format(
            yml['field'].replace('HATLAS-',''), 
            yml['field'].lower(), 
            yml['final'].split('.')[0].split('_')[-1] )

'../dmu32_EGS/data/depths_egs_20180501.fits'

In [11]:
depths

field,ferr_u_mean_median,ferr_g_mean_median,ferr_r_mean_median,ferr_i_mean_median,ferr_z_mean_median,ferr_y_mean_median,ferr_j_mean_median,ferr_h_mean_median,ferr_k_mean_median,ferr_ks_mean_median,ferr_irac_i1_mean_median,ferr_irac_i2_mean_median,ferr_irac_i3_mean_median,ferr_irac_i4_mean_median,ferr_mips_24_mean_median,ferr_pacs_green_mean_median,ferr_pacs_red_mean_median,ferr_spire_250_mean_median,ferr_spire_350_mean_median,ferr_spire_500_mean_median
str18,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
COSMOS,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
ELAIS-S1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
GAMA-09,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Lockman-SWIRE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
ELAIS-N2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Herschel-Stripe-82,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
SA13,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
GAMA-12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
GAMA-15,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [12]:
depths.write('dr1_depth_overviews.csv')

## Make a single all sky order ten depth map

In [2]:
depth_files=glob.glob('./data/depth*.fits')

In [3]:
not True

False

In [13]:
full_depth = Table()
for f in depth_files:
    d_cat = Table.read(f)
    for c in d_cat.colnames:
        if not((c == 'hp_idx_O_10') or (c.endswith('mean'))):
            d_cat.remove_column(c)
        else:
            pass
        
    u_cat = unique(d_cat,keys='hp_idx_O_10')
    full_depth = vstack([full_depth,u_cat])
full_depth = full_depth.filled()
full_depth.write('./data/all_sky_depth_map.fits',overwrite=True)

In [26]:
full_depth[:5]

hp_idx_O_10,ferr_ap_decam_g_mean,ferr_ap_decam_r_mean,ferr_ap_decam_i_mean,ferr_ap_decam_z_mean,ferr_ap_decam_y_mean,ferr_ap_vista_j_mean,ferr_ap_vista_h_mean,ferr_ap_vista_ks_mean,ferr_ap_irac_i1_mean,ferr_ap_irac_i2_mean,ferr_ap_g_mean,ferr_ap_r_mean,ferr_ap_i_mean,ferr_ap_z_mean,ferr_ap_y_mean,ferr_ap_j_mean,ferr_ap_h_mean,ferr_ap_ks_mean,ferr_ap_gpc1_g_mean,ferr_ap_gpc1_r_mean,ferr_ap_gpc1_i_mean,ferr_ap_gpc1_z_mean,ferr_ap_gpc1_y_mean
,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,,,,,,,,,uJy,uJy,uJy,uJy,uJy
int64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64
8687019,0.08941301241009876,0.10159930664410594,0.16957028237319402,0.3360431102221077,1.1923333949792894,1.8040026,nan,5.3524575,nan,nan,0.14499704199338645,0.15886723307782735,0.2912562074742357,0.5985401711559398,2.082680118606706,4.5748534,nan,13.370395,nan,nan,nan,nan,nan
8687021,0.10287259124016224,0.12039300871488914,0.18888852873625897,0.3987357319469088,1.419343910221842,1.8775079,nan,5.459472,nan,nan,0.15923118742553097,0.1769440101324317,0.26167305703021043,0.6090910253657126,2.0849519539507115,2.9474516,nan,10.134894,nan,nan,nan,nan,nan
8687022,0.09532933268539538,0.10906805032746564,0.18241326357123988,0.35396133156307974,1.2651862089775676,1.921677,nan,5.4809356,nan,nan,0.14891398565063033,0.1715184602265481,0.32075134581481485,0.6570133593223575,2.509728964865259,5.170397,nan,16.788235,nan,nan,nan,nan,nan
8687023,0.10824620420702666,0.12451767176397369,0.20268880115737872,0.3935921662498461,1.4397059241818613,2.5650663,nan,6.3428183,nan,nan,0.16647172217423,0.19354728602276483,0.3406217292725189,0.7090275556854875,2.75275276304709,5.436189,nan,14.047213,nan,nan,nan,nan,nan
8687027,0.09558850799653333,0.10926661967339074,0.18378175455724388,0.3761169368503513,nan,nan,nan,nan,nan,nan,0.2635383775437744,0.31662367450327605,0.5523959165758974,1.1465177965441924,nan,nan,nan,nan,nan,nan,nan,nan,nan
8687030,0.1358427699340554,0.1493070859514625,0.2240513508077266,0.4582541478237105,1.6245409469711032,2.7268982,nan,6.1417556,nan,nan,0.18002077580602077,0.1959855108897479,0.3441068031171864,0.8023753186870656,2.9411182516985024,5.4317064,nan,13.961416,nan,nan,nan,nan,nan
8687031,0.10741622498099045,0.11990133701929664,0.19915376209503743,0.3899759458666783,1.4604985733744726,2.254099,nan,5.835954,nan,nan,0.15703056056224338,0.18470731440286034,0.3271232713991038,0.6792248384456054,2.5561633557780086,4.47669,nan,12.344089,nan,nan,nan,nan,nan
8687032,0.1063021599962953,0.11882482910862704,0.19231578010450898,0.381007675223252,1.3052594731374605,1.9479681,nan,5.5365825,nan,nan,0.15411750281011016,0.1796824392500375,0.31161490019650584,0.6490385043926964,2.415877941023494,4.6447783,nan,13.411079,nan,nan,nan,nan,nan
